In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

**8. This problem involves the OJ data set which is part of the ISLR
package.
372 9. Support Vector Machines
(a) Create a training set containing a random sample of 800
observations, and a test set containing the remaining
observations.**

In [3]:
OJ = pd.read_csv('Data/OJ.csv').drop('Unnamed: 0', axis=1).dropna()
OJ.head()

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


In [4]:
from sklearn.model_selection import train_test_split
columns_value = '0'
for column_value in OJ.columns:
    columns_value = columns_value + '+' + column_value
#Store7和STORE共线性
y,X = patsy.dmatrices('Purchase ~ {} - Purchase - Store7'.format(columns_value), data = OJ, return_type="dataframe")
X_train, X_test = train_test_split(X, train_size=800, random_state=1988)
y_train, y_test = train_test_split(y, train_size=800, random_state=1988)

**(b) Fit a support vector classifier to the training data using
cost=0.01 , with Purchase as the response and the other variables
as predictors. Use the summary() function to produce summary
statistics, and describe the results obtained.**

In [5]:
from sklearn.svm import SVC
svc = SVC(kernel="linear",C = 0.01)
svc.fit(X_train ,y_train['Purchase[CH]'])

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [6]:
svc.n_support_

array([303, 305])

In [7]:
svc.classes_

array([0., 1.])

含有608个支持向量，其中303个属于MM，305个属于CH

**(c) What are the training and test error rates?**

In [8]:
(svc.predict(X_train) != y_train['Purchase[CH]']).sum()/len(X_train)

0.2025

In [9]:
(svc.predict(X_test) != y_test['Purchase[CH]']).sum()/len(X_test)

0.25555555555555554

**(d) Use the tune() function to select an optimal cost . Consider val-
ues in the range 0.01 to 10.**

In [10]:
from sklearn.model_selection import GridSearchCV
svm = SVC(kernel="linear")
param_grid = [
{'C':10 ** np.linspace(-2,0,3) },
]
grid_search = GridSearchCV(svm , param_grid, cv = 10, scoring='accuracy',return_train_score=True)
grid_search.fit(X_train,y_train['Purchase[CH]'])

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': array([0.01, 0.1 , 1.  ])}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='accuracy', verbose=0)

In [11]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(1 - mean_score, params)

0.22124999999999995 {'C': 0.01}
0.17125 {'C': 0.1}
0.15874999999999984 {'C': 1.0}


In [12]:
grid_search.best_params_

{'C': 1.0}

**(e) Compute the training and test error rates using this new value
for cost .**

In [13]:
svc = SVC(kernel="linear",C = 1)
svc.fit(X_train ,y_train['Purchase[CH]'])

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [14]:
svc.n_support_

array([160, 162])

In [15]:
(svc.predict(X_train) != y_train['Purchase[CH]']).sum()/len(X_train)

0.1525

In [16]:
(svc.predict(X_test) != y_test['Purchase[CH]']).sum()/len(X_test)

0.2074074074074074

**(f) Repeat parts (b) through (e) using a support vector machine
with a radial kernel. Use the default value for gamma .**

In [17]:
svm_rbf = SVC(kernel="rbf",C = 0.01,gamma  = 'auto')
svm_rbf.fit(X_train ,y_train['Purchase[CH]'])

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:
svm_rbf.n_support_

array([308, 322])

含有630个支持向量，其中308个属于MM，322个属于CH

In [19]:
(svm_rbf.predict(X_train) != y_train['Purchase[CH]']).sum()/len(X_train)

0.385

In [20]:
(svm_rbf.predict(X_test) != y_test['Purchase[CH]']).sum()/len(X_test)

0.40370370370370373

In [21]:
svm = SVC(kernel="rbf",gamma  = 'auto')
#与R不同，这里C需要大一点
param_grid = [
{'C':10 ** np.linspace(0,3,4) },
]
grid_search = GridSearchCV(svm , param_grid, cv = 10, scoring='accuracy',return_train_score=True)
grid_search.fit(X_train,y_train['Purchase[CH]'])

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': array([   1.,   10.,  100., 1000.])}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='accuracy', verbose=0)

In [22]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(1 - mean_score, params)

0.26249999999999996 {'C': 1.0}
0.18249999999999988 {'C': 10.0}
0.17375000000000007 {'C': 100.0}
0.1875 {'C': 1000.0}


In [23]:
grid_search.best_params_

{'C': 100.0}

In [24]:
svm_rbf = SVC(kernel="rbf",C = 100,gamma  = 'auto')
svm_rbf.fit(X_train ,y_train['Purchase[CH]'])

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [25]:
(svm_rbf.predict(X_train) != y_train['Purchase[CH]']).sum()/len(X_train)

0.09375

In [26]:
(svm_rbf.predict(X_test) != y_test['Purchase[CH]']).sum()/len(X_test)

0.26666666666666666

**(g) Repeat parts (b) through (e) using a support vector machine
with a polynomial kernel. Set degree=2.**

In [27]:
svm_poly = SVC(kernel="poly",C = 0.01,gamma  = 'auto',degree = 2)
svm_poly.fit(X_train ,y_train['Purchase[CH]'])

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [28]:
svm_poly.n_support_

array([152, 152])

In [29]:
(svm_poly.predict(X_train) != y_train['Purchase[CH]']).sum()/len(X_train)

0.14875

In [30]:
(svm_poly.predict(X_test) != y_test['Purchase[CH]']).sum()/len(X_test)

0.2111111111111111

In [31]:
svm = SVC(kernel="poly",gamma  = 'auto',degree = 2)
#这里C太大会运行太慢
param_grid = [
{'C': 2 * 10 ** np.linspace(-5,-2,4) },
]
grid_search = GridSearchCV(svm , param_grid, cv = 10, scoring='accuracy',return_train_score=True)
grid_search.fit(X_train,y_train['Purchase[CH]'])

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=2,
                           gamma='auto', kernel='poly', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': array([2.e-05, 2.e-04, 2.e-03, 2.e-02])}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='accuracy', verbose=0)

In [32]:
grid_search.best_params_

{'C': 0.002}

In [33]:
svm_poly = SVC(kernel="poly",C = 0.002,gamma  = 'auto',degree = 2)
svm_poly.fit(X_train ,y_train['Purchase[CH]'])

SVC(C=0.002, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [34]:
(svm_poly.predict(X_train) != y_train['Purchase[CH]']).sum()/len(X_train)

0.15125

In [35]:
(svm_poly.predict(X_test) != y_test['Purchase[CH]']).sum()/len(X_test)

0.2074074074074074

**(h) Overall, which approach seems to give the best results on this
data?**

degree = 2的多项式核函数